#FirstName LastName (SIS ID#)

# ECI 115: Homework 3

## <font color='red'>**Problem 24.16**</font>

Perform the same computation as in Sec. 24.2, but use $O(h^8)$ Romberg integration to evaluate the integral:
$$F=\int_{0}^{30}200\Big(\frac{z}{5+z}\Big)e^{-2z/30}dz$$

<font color='red'> **TL;DR**:
Explain how to compute the $O(h^8)$ Romberg integration. Create a program that uses this method to integrate the function above. Print the result. Discuss the accuracy of this result and how many function evaluations were required to achieve this accuracy.
</font>

### <font color='blue'>**Solution 24.16**</font>

In [22]:
import numpy as np
def f(z):
  return 200 * (z/(5+z)) * np.exp(-2*z/30)

def trapezoid(func,a: float,b: float,n_panels: int):
  x = np.linspace(a, b, n_panels + 1)
  y = func(x)
  h = (b - a) / n_panels
  return h * (0.5 * y[0] + y[1:-1].sum() + 0.5 * y[-1])

def romberg(func, a, b, total=1e-8, max=10):
    # estimate integral of f(x) from a to b
    # stop when tolerance reached or maxiter exceeded
  R = np.zeros((max, max))  # Romberg matrix
  n = 1 # first step size: (b-a), one segment
  R[0,0] = trapezoid(func,a,b,n)
    # Add a new row every iteration, and update columns
  for iter in range(1, max):
        n = n * 2 # every new row uses 2x the segments for the trapezoid rule
        R[iter, 0] = trapezoid(func,a,b,n)
        # update all other columns
        for k in range(1,iter+1):
            j = iter - k # Indexing hack to place the new value
            R[j,k] = (4**k * R[j+1,k-1] - R[j,k-1]) / (4**k - 1)


            if abs(R[0,k] - R[0,k-1]) < total:
              break
  I = R[0,k]
  num_evals = n+1
  return I, num_evals

a = 0
b = 30
I, num_evals = romberg(f, a, b, max=4)
print('Romberg 1: I=%0.7f, num_evals = %d' % (I, num_evals))



Romberg 1: I=1476.7972937, num_evals = 9


**Discussion of Results**

[The O(h^8) Romberg iintegration is going ot first start with composite trapezoid rule on [a,b] using a sequence of step sizes that half each time.  The step size is going to be calculated with h = (b-a) / n and this helps to produce the trapezoid estimates Ri,0 for i=0,1,2, etc.  Romberg integration also applies Richardson extrapolation to eliminate any errors from the error expansion involving even powers of h.  The extrapolation formula is used to produce increasingly accurate estimates along each row of the Romberg table. In order to reach O(h^8) accuracy, the method has to build the table through the third diagonal entry which would require four levels of trapezoid-rule refinement.  The final diagonal value is going to serve as the high-accuracy approximation of the definite integral.

Using this Romberg procedure, we were able to get the value of I = 1476.797 and this value is highly accurate.  The reason for that is because the lower-rder errors were removed through extrapolation.  To reach this accuracy, the finest trapezoid refinement used 8 panels, which required the function to be evaluated at 9 points.  What this shows us is that we can achieve high accuracy with a small number of function evaluations.]

## <font color='red'>**Problem 24.17**</font>

Perform the same computation as in Sec. 24.2, but use Gauss quadrature to evaluate the integral:
$$\int_{0}^{30}200\Big(\frac{z}{5+z}\Big)e^{-2z/30}dz$$


<font color='red'> **TL;DR**:
Explain how the Gauss Quadrapture works. Create a program that uses this method to integrate the function above. Print results for using the Gauss Quadrapture for 2, 3, 4, 5, and 6 points. See Table 22.1 below. Discuss how the accuracy of your result changes as your increase the number of points used to calculate the Gauss quadrature.
</font>

<p align="center">
  <img src="https://github.com/cdefinnda/ECI-115_HW-Images/blob/main/HW3_Table22.1.png?raw=true" alt="Table 22.1" width=500>
</p>

### <font color='blue'>**Solution 24.17**</font>

In [25]:
# define integration points, in this case 0 through 30
a = 0
b = 30
# two-point Gauss quadrature
def gauss2(func,a,b):
    c = np.array([1,1])
    x = np.array([-1/np.sqrt(3), 1/np.sqrt(3)])

    y = (a+b)/2 + (b-a)/2 * x  # map from [-1,1] to [a,b]
    Answ = (b-a)/2 * sum(c * f(y))  # estimate integral
    return Answ
Answ = gauss2(f, a, b)
print(Answ)


1610.5722652945797


**Discussion of Results**

The Gauss-Legendre quadrature estimates an integral by sampling the function at a few carefully chosen points and multiplying those values by specific weights.  Those nodes and weights are selected so that the method is exact for all polynomials up to degree 2n-1 which helps to make sure that it is very accurate. In fact it makes sure it is accurate even when using only a small number of evaluations.  Because Gauss quadrature is defined on interval [-1,1], any integralon [a,b] is converted to this standard interval using a change of variables.  After mapping the nodes into [a,b], the weighted sum of the function values gives an approximation of the integral.  

Using only two Gauss points, the estimate was 1610.57 which is quite higher than more accurate reference value of 1476.80.  The reason this happens is we are not able to capture the shape of the function very well over the long interval [0,30] by using only two points.  When we increase the number of Gauss points to 3,4,5, or 6, the method becomes much more accurate becausethe method is able to check the function at more positions across the interval  Because that would use more information about the curve, the estimate becomes closer to the true value.  

## <font color='red'>**Problem 12.14**</font>

A civil engineer invovled in construction requires 4800, 5800, and 5700 m$^3$ of sand, fine gravel, and coarse gravel, respectively, for a building project. There are three pits from which these materials can be obtained. The composition of these pits is

|       | Sand % | Fine Gravel % | Coarse Gravel % |
|:-----:|:------:|:-------------:|:---------------:|
| Pit 1 |   52   |       30      |        18       |
| Pit 2 |   20   |       50      |        30       |
| Pit 3 |   25   |       20      |        55       |

How many cubic meters must be hauled from each pit in order to meet the engineer's needs?

<font color='red'> **TL;DR**:
Explain how to convert the information in the problem above into a system of equations that can be solved using linear algebra. Create a program that computes the volumes from pits 1, 2, and 3. Print these results. Discuss what this result means physically.
</font>

### <font color='blue'>**Solution 12.14**</font>



In [6]:
import numpy as np

a_percentages = np.array([[0.52, 0.2, 0.25], [0.3, 0.5, 0.2], [0.18, 0.3, 0.55]])
b_masses = np.array([4800, 5800, 5700])

x_mass_percentages = np.linalg.solve(a_percentages,b_masses)

print('Mass of each material (m^3): ',x_mass_percentages,)
print('Total Mass needed (m^3): ',sum(x_mass_percentages))

Mass of each material (m^3):  [4005.81395349 7131.39534884 5162.79069767]
Total Mass needed (m^3):  16300.0


**Discussion of Results**

To turn the problem into equations, we let x1,x2, and x3 be the amounts taken from Pits 1,2, and 3.  Each pit is going to have a certain percentage of sand, fine gravel, and coarse gravel, so the amount of each material that we get is the percentage times the amount hauled.  For example, Pit 2 gives 20% sand, so the san contributed by Pit 2 is: 0.20x2 and for Pit 1, it gives 52% sand so it'll be 0.52x1.  We do this for pit 3 as well and match each material to the required totals of 4800, 5800, and 5700m^3 which will give us three equations.  These three equations will form a linear system that can be solved to find out how much material to take from each pit.

What the result means physicall is that it tells us how many cubic meters to haul from each pit so that the final mixture contains the right amount of sand, fine gravel, and coarse gravel.  What this means physically is that we are combining material from several pits to meet the engineers needs.  Because each pit gives a mix of the materials, the engineer must haul more total volume to make sure that the correct amounts of each material add up.

## <font color='red'>**Problem 12.17**</font>

Calculate the forces and reactions for the truss in Fig. 12.4 if a downward force of 3,000 kg and a horizontal force to the right of 1500 kg are applied at node 1.

**Figure 12.4**. Forces on a statically determinate truss.
<p align="center">
  <img src="https://github.com/cdefinnda/ECI-115_HW-Images/blob/main/HW3_Fig.12.4.png?raw=true" alt="Fig. 12.4" width=500>
</p>

<font color='red'> **TL;DR**:
Explain how force balances can be used to convert the information in the problem above into a system of equations that can be solved using linear algebra, and how LU Decomposition could be used to solve this problem. Create a program that solves for the unknowns ($F_1$, $F_2$, $F_3$, $H_2$, $V_2$, and $V_3$). Print these results. Discuss the advantages of LU Decomposition over Gauss Elimination.
</font>

### <font color='blue'>**Solution 12.17**</font>



In [5]:
import numpy as np
coef = np.array([[0.866025, -0.5, 0, 0, 0, 0,],
                  [0.5, 0.866025, 0, 0, 0, 0],
                  [-0.866025, 0, -1, 0, -1, 0],
                  [-0.5, 0, 0, -1, 0, 0],
                  [0, 0.5, 0, 0, 1, 0],
                  [0, -0.866025, 0, 0, 0, -1]])
W1v = -3000 #kg
W1h = 1500 #kg
b = [W1h, W1v, 0, 0, 0, 0]
n = coef.shape[0]
U = coef.copy()
L = np.eye(n)
# solve with LU decomposition or np.lingalg.solve()

for k in range(n-1):
  for i in range(k+1, n):
    L[i,k] = U[i,k] / U[k,k]
    U[i,:] = U[i,:] - L[i,k] * U[k,:]
d = np.zeros(n)
d[0] = b[0] / L[0,0]

for i in range(1,n):
  d[i] = (b[i] - L[i,:] @ d) / L[i,i]

# use back substitution
x = np.zeros(n)
x[-1] = d[-1] / U[-1,-1] # index -1 for last element

for i in range(n-2,-1,-1):
  x[i] = (d[i] - np.dot(U[i, i+1:n], x[i+1:n])) / U[i, i]
print('F1:', x[0])
print('F2:', x[4])
print('F3:', x[1])
print('H2:', x[2])
print('V2:', x[3])
print('V3:', x[5])

F1: -200.96264054824653
F2: 1674.0386707807952
F3: -3348.0773415615904
H2: -1500.0
V2: 100.48132027412339
V3: 2899.5186797258766


**Discussion of Results**

To set up the system of equations, we apply force balance at each joint of the truss using the conditions of sum of Fx and Fy are equal to zero.  Each member force contributes a horizontal and vertical component based on its angle, and the reaction forces also act in specific directions. Writing these force–balance equations at nodes 1, 2, and 3 gives us six equations with six unknown forces.  These equations can then be written in matrix form so they can be solved using linear algebra. LU decomposition solves this system by breaking the matrix into two simpler matrices so we can first use forward substitution and then back substitution to find all the unknown forces.

LU decomposition has several advantages over Gauss elimination. LU decomposition is faster because the matrix is factored only once into a lower-triangular matrix L and an upper-triangular matrix U.  After this, solving the system becomes much faster because we only need forward and back substitution.  In Gauss elimination on the other hand, you must redo the entire elimination process every time you solve the system which can be slow.